In [2]:
import numpy as np

def is_block_triangular(A, tol=1e-10):
    """Check if matrix A is block upper or lower triangular.
    Returns: (kind, sizes) where kind is 'upper' or 'lower', sizes is list of block sizes.
    """
    n = A.shape[0]
    # Try upper triangular
    for i in range(n):
        for j in range(i+1, n):
            if np.abs(A[j,i]) > tol:
                break
        else:
            continue
        break
    else:
        # Found block structure for upper
        sizes = []
        start = 0
        for i in range(1, n+1):
            if i == n or np.any(np.abs(A[i:,:i]) > tol):
                sizes.append(i - start)
                start = i
        return 'upper', sizes

    # Try lower triangular
    for i in range(n):
        for j in range(i):
            if np.abs(A[j,i]) > tol:
                break
        else:
            continue
        break
    else:
        # Found block structure for lower
        sizes = []
        start = 0
        for i in range(1, n+1):
            if i == n or np.any(np.abs(A[:i,i:]) > tol):
                sizes.append(i - start)
                start = i
        return 'lower', sizes

    return None, None

def make_block_triangular_with_permutation(A, tol=1e-10):
    """Convert matrix A to block upper triangular form using row/column swaps.
    Returns: (A_block, P) where A_block is the block triangular matrix,
             and P is the permutation matrix that was applied.
    """
    n = A.shape[0]
    A_copy = A.copy()
    P = np.eye(n)  # Start with identity permutation matrix

    # We'll try to make it block upper triangular by processing diagonal blocks
    i = 0
    while i < n:
        # Find the size of the current diagonal block
        # Look for the first non-zero element below the diagonal in column i
        j = i
        while j < n and np.allclose(A_copy[j:i, i], 0, atol=tol):
            j += 1

        if j == n:
            # No more elements, we're done
            break

        # The block size is from i to j
        block_size = j - i

        # If there are any non-zeros above the diagonal in this block,
        # we need to swap rows/columns to eliminate them
        # But since we want upper triangular, we actually don't need to do anything
        # because we're processing from top-left to bottom-right

        # Move to next block
        i = j

    # Now we have a block structure, but we might need to permute to make it strictly block upper triangular
    # We can use a simple greedy algorithm: for each position (i,j) with i>j, if A[i,j] != 0,
    # we swap rows and columns to move this element out of the lower triangle

    # Actually, let's use a different approach: find a permutation that makes the matrix block upper triangular
    # This is equivalent to finding a permutation such that the matrix becomes block upper triangular

    # We'll use a greedy algorithm to build the block structure
    # Start with the original matrix
    A_final = A_copy.copy()
    P_final = np.eye(n)

    # Process from top-left to bottom-right
    i = 0
    while i < n:
        # Check if there are any non-zero elements below the diagonal in column i
        has_below = False
        for k in range(i+1, n):
            if np.abs(A_final[k, i]) > tol:
                has_below = True
                break

        if not has_below:
            # Move to next column
            i += 1
            continue

        # Find the first row k > i where A_final[k, i] != 0
        k = i + 1
        while k < n and np.abs(A_final[k, i]) <= tol:
            k += 1

        if k >= n:
            # Shouldn't happen, but just in case
            break

        # Swap rows i and k
        A_final[[i, k]] = A_final[[k, i]]
        P_final[[i, k]] = P_final[[k, i]]

        # Also swap columns i and k (since we want to preserve the block structure)
        A_final[:, [i, k]] = A_final[:, [k, i]]
        P_final[:, [i, k]] = P_final[:, [k, i]]

        # Now check again for the same column
        # Since we swapped, we need to check if there are still non-zeros below
        # But we only care about making it block upper triangular, so we continue

        # Note: This is a simplified version. In practice, you might want to use a more sophisticated
        # algorithm to find the optimal block structure.

        # For now, let's assume we've made progress and continue
        i += 1

    # After this process, A_final should be block upper triangular
    # Let's verify and get the block sizes
    kind, sizes = is_block_triangular(A_final, tol)

    if kind is None:
        # If we couldn't make it block triangular, return the best we could
        # For simplicity, we'll return what we have
        return A_final, P_final, None, None

    return A_final, P_final, kind, sizes

#===== Demo Cell =====
if __name__ == "__main__":
    # Example matrix
    A = np.array([[2, 1, 3],
                  [0, 4, 5],
                  [0, 0, 6]], dtype=float)

    print("Original Matrix A:")
    print(A)

    # Convert to block triangular
    A_block, P, kind, sizes = make_block_triangular_with_permutation(A)

    print(f"\nBlock {kind} triangular matrix:")
    print(A_block)

    print(f"\nPermutation matrix P:")
    print(P)

    print(f"\nBlock sizes: {sizes}")

    # Verify: P^T * A * P should give us the block triangular form
    # (Note: In our implementation, we applied the same row and column swaps,
    # which corresponds to P^T * A * P)
    A_check = P.T @ A @ P
    print(f"\nVerification (P^T * A * P):")
    print(A_check)

    # Check if they are close
    print(f"\nAre A_block and P^T * A * P close? {np.allclose(A_block, A_check)}")

Original Matrix A:
[[2. 1. 3.]
 [0. 4. 5.]
 [0. 0. 6.]]

Block upper triangular matrix:
[[2. 1. 3.]
 [0. 4. 5.]
 [0. 0. 6.]]

Permutation matrix P:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Block sizes: [3]

Verification (P^T * A * P):
[[2. 1. 3.]
 [0. 4. 5.]
 [0. 0. 6.]]

Are A_block and P^T * A * P close? True
